In [1]:
from datetime import date
from highlight_text import fig_text
from pathlib import Path

import os
import pandas as pd
import matplotlib.pyplot as plt
import bigframes.pandas as bpd

import warnings
warnings.filterwarnings(action='once')

%load_ext google.cloud.bigquery
bpd.options.bigquery.project = "dhh-ncr-stg"

/Users/patrick.doupe/DH/ad_hoc/cohorts/.venv/lib/python3.13/site-packages/google/cloud/bigquery/__init__.py:237: FutureWarning: %load_ext google.cloud.bigquery is deprecated. Install bigquery-magics package and use `%load_ext bigquery_magics`, instead.
  warnings.warn(


## Open questions

1. how can we flag vendor and chain?
2. how can we get hex data, or something nice and local
3. any other data that we're missing?


TODO
- want cross validation through time
- 


In [2]:
# Get the current script's directory
CURRENT_DIR = Path(os.getcwd())
# Navigate to parallel directory and read the SQL file
BASE_SQL_FILE_PATH = CURRENT_DIR.parent / "queries" / "data_creation.sql"
CURRENT_COHORT_SQL_FILE_PATH = CURRENT_DIR.parent / "queries" / "current_cohorts.sql"
CURRENT_RECOMMENDATIONS_SQL_FILE_PATH = CURRENT_DIR.parent / "queries" / "current_recommendations.sql"

In [3]:
with open(BASE_SQL_FILE_PATH, 'r') as file:
    SQL_CONTENT = file.read()

In [ ]:
%%bigquery
$SQL_CONTENT

Executing query with job ID: e7a5fa11-e1ca-4abc-96fe-56e7fb49fa25
Query executing: 0.45s

In [ ]:
with open(CURRENT_COHORT_SQL_FILE_PATH, 'r') as file:
    SQL_CONTENT_CURRENT = file.read()

In [ ]:
%%bigquery
$SQL_CONTENT_CURRENT

/Users/patrick.doupe/DH/ad_hoc/cohorts/.venv/lib/python3.13/site-packages/google/cloud/bigquery/job/query.py:2074: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  query_result = wait_for_query(self, progress_bar_type, max_results=max_results)


""


In [ ]:
%%bigquery
SELECT COUNT(*) FROM `dhh-ncr-stg.patrick_doupe.current_cohort_vendor_base`

/Users/patrick.doupe/DH/ad_hoc/cohorts/.venv/lib/python3.13/site-packages/google/cloud/bigquery/job/query.py:2074: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  query_result = wait_for_query(self, progress_bar_type, max_results=max_results)
/Users/patrick.doupe/DH/ad_hoc/cohorts/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:2645: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,f0_
0,3050846


In [ ]:
%%bigquery
DECLARE REPORT_DATE date DEFAULT '2025-06-01';
DECLARE target_entities ARRAY<STRING>; 
SET target_entities = ['FP_SG', 'TB_AE']; 

SELECT 
    COUNT(DISTINCT vendor_code)
FROM `fulfillment-dwh-production.cl_vendor.growth_vendor_smart_recommendations`
WHERE created_date = REPORT_DATE 
AND entity_id IN UNNEST(target_entities)

/Users/patrick.doupe/DH/ad_hoc/cohorts/.venv/lib/python3.13/site-packages/google/cloud/bigquery/job/query.py:2074: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  query_result = wait_for_query(self, progress_bar_type, max_results=max_results)
/Users/patrick.doupe/DH/ad_hoc/cohorts/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:2645: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,f0_
0,50390


In [ ]:
%%bigquery
SELECT * FROM `dhh-ncr-stg.patrick_doupe.current_cohort_vendor_base`
LIMIT 10

/Users/patrick.doupe/DH/ad_hoc/cohorts/.venv/lib/python3.13/site-packages/google/cloud/bigquery/job/query.py:2074: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  query_result = wait_for_query(self, progress_bar_type, max_results=max_results)
/Users/patrick.doupe/DH/ad_hoc/cohorts/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:2645: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,global_entity_id,vendor_id,cohort_id,created_month,successful_orders,successful_customers,successful_orders_gmv
0,TB_BH,697347,208,NaT,<NA>,<NA>,None
1,TB_AE,613389,1598,NaT,<NA>,<NA>,None
2,TB_AE,637426,2996,NaT,<NA>,<NA>,None
3,TB_AE,689954,4606,NaT,<NA>,<NA>,None
4,TB_AE,695929,6858,NaT,<NA>,<NA>,None
5,TB_AE,42236,8771,NaT,<NA>,<NA>,None
6,TB_AE,4582,8822,NaT,<NA>,<NA>,None
7,TB_AE,721359,8830,NaT,<NA>,<NA>,None
8,TB_AE,712149,8989,NaT,<NA>,<NA>,None
9,TB_AE,705049,10184,NaT,<NA>,<NA>,None


## Current recommendations

In [ ]:
with open(CURRENT_RECOMMENDATIONS_SQL_FILE_PATH, 'r') as file:
    SQL_CONTENT = file.read()

In [ ]:
%%bigquery
$SQL_QUERY